<a href="https://colab.research.google.com/github/karolinablaszka/Project-ML/blob/EDA/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.covariance import EllipticEnvelope

In [5]:
df = pd.read_csv('train_data.csv',  on_bad_lines='skip')

In [3]:
df.head()

,304.361,-436.590,-33854.769,-96571.569,-15086.947,-794.409,-442.859,304.248,-202.411,-506.852,...,95.294,-3.936,-251.812,256.429,-473.661,-1398.604,42.638,771.185,-191.123,1356.137
0,54.047,90.087,-154.747,-4116.486,38365.133,-589.309,1115.367,274.199,814.953,167.272,...,-176.341,-426.238,-542.428,99.147,-102.309,-277.948,-98.859,-1163.779,265.231,-992.056
1,272.168,-201.736,4212.592,-9123.655,-35422.458,-1029.249,-506.290,-476.856,-570.731,-161.288,...,321.426,-376.033,142.834,920.958,-122.525,-174.304,-137.612,-1571.473,678.323,1020.263
2,170.007,68.736,48662.079,29735.235,-13903.955,895.081,-257.748,-811.058,-691.561,-31.439,...,-409.919,400.946,313.270,123.172,1786.962,147.637,31.433,-784.719,573.433,1454.415
3,86.797,180.052,-49643.545,2515.406,26206.315,-407.453,-189.416,-53.664,-159.507,-42.291,...,-101.761,-424.898,37.254,-337.431,423.691,14.240,267.352,-234.560,-213.804,873.391
4,345.239,-190.663,-19609.472,-44312.301,-3143.034,-681.815,-711.535,-1358.550,396.870,-7.566,...,-28.348,227.120,2.236,70.115,312.647,491.650,184.906,-46.391,286.278,53.120


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Columns: 10000 entries, 304.361 to 1356.137
dtypes: float64(10000)
memory usage: 9.5 MB


In [8]:
df.describe()

,304.361,-436.590,-33854.769,-96571.569,-15086.947,-794.409,-442.859,304.248,-202.411,-506.852,...,95.294,-3.936,-251.812,256.429,-473.661,-1398.604,42.638,771.185,-191.123,1356.137
count,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,...,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000,3749.000000
mean,30.571124,9.199832,178.755026,837.059166,-182.034161,25.936548,25.042892,18.985189,14.883948,15.109637,...,13.975692,21.463356,22.659653,16.672954,14.122260,-1.053665,9.588436,-2.237489,-2.791015,37.720326
std,286.175656,213.070740,43667.290847,57108.922663,21944.043101,711.525120,898.196470,868.779430,532.786491,236.627576,...,269.255977,545.819172,265.218686,503.468371,818.895883,655.356364,204.758153,793.027599,629.205060,768.206519
min,-989.450000,-859.922000,-163336.428000,-243844.632000,-77566.109000,-2414.196000,-3227.681000,-3151.321000,-1792.781000,-794.271000,...,-957.937000,-1983.009000,-937.357000,-1758.858000,-2526.791000,-2165.769000,-739.205000,-2731.037000,-2223.909000,-2565.513000
25%,-163.862000,-135.333000,-29259.560000,-38027.475000,-14405.498000,-448.189000,-590.534000,-563.235000,-349.827000,-144.145000,...,-169.789000,-352.960000,-154.630000,-328.286000,-550.677000,-431.826000,-129.134000,-561.480000,-424.420000,-462.913000
50%,35.463000,2.744000,1208.590000,1037.514000,-147.659000,14.138000,13.895000,16.537000,9.733000,11.226000,...,18.700000,38.426000,26.220000,14.567000,1.836000,-3.789000,8.363000,-4.317000,-7.446000,27.562000
75%,215.740000,156.269000,29640.573000,40569.974000,14526.865000,507.316000,631.339000,608.810000,383.490000,179.828000,...,196.115000,396.910000,198.267000,356.267000,574.552000,425.535000,149.917000,538.809000,427.864000,552.124000
max,1224.394000,722.868000,177744.241000,215372.854000,86287.955000,2446.466000,3118.087000,2966.971000,1914.308000,803.200000,...,893.196000,2080.750000,1070.453000,1705.664000,2868.989000,2562.331000,685.176000,3123.617000,2204.527000,2664.074000


In [5]:
df.isnull().sum().sum() 

7704

In [6]:
df.isnull().sum().sort_values()

304.361     0
-468.585    0
-110.546    0
209.747     0
-236.969    0
           ..
1527.575    1
26.314      1
-501.133    1
365.950     1
1356.137    1
Length: 10000, dtype: int64

In [7]:
df.dropna(axis=1, how='any', inplace=True)

In [9]:
df.drop_duplicates(subset=None, keep='first', inplace=True)

In [11]:
df.shape

(125, 2296)

In [12]:
df.nunique()

304.361       125
-436.590      125
-33854.769    125
-96571.569    125
-15086.947    125
             ... 
1140.654      125
562.469       125
-131.422      125
-82.341       125
647.231       125
Length: 2296, dtype: int64

In [13]:
# corelation = df.corr()

In [ ]:
# sns.heatmap(corelation, xticklabels=corelation.columns, yticklabels=corelation.columns,
#           annot=True)

In [ ]:
outlier_detector = EllipticEnvelope(contamination=.1)

outlier_detector.fit(df)

outlier_detector.predict(df)